# XGBoost

## Imports

In [93]:
%pip install xgboost
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder, StandardScaler, RobustScaler
import numpy as np
import hashlib

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


## Dataset

In [ ]:
dataset_path = './dataset/data/NF-UQ-NIDS-v2.csv'
sample_size = 1000000
data = pd.read_csv(dataset_path, nrows=sample_size)
# data.to_csv('sample.csv', index=False)
# print(data.iloc[0])

features = data.drop(['Attack', 'IPV4_SRC_ADDR', 'IPV4_DST_ADDR'], axis=1)
target = data['Attack']

non_numeric_cols = features.select_dtypes(include=['object']).columns

encoder = LabelEncoder()
for col in non_numeric_cols:
    features[col] = encoder.fit_transform(features[col])

target = encoder.fit_transform(target)

numeric_cols = features.select_dtypes(include=[np.number]).columns
scaler = RobustScaler()
features[numeric_cols] = scaler.fit_transform(features[numeric_cols])

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)


AttributeError: 'int' object has no attribute 'encode'

## Model

In [90]:
model = xgb.XGBClassifier(eval_metric='logloss')

## Training

In [91]:
model.fit(X_train, y_train)

XGBoostError: [10:52:33] /workspace/src/data/../common/../data/gradient_index.h:94: Check failed: valid: Input data contains `inf` or a value too large, while `missing` is not set to `inf`
Stack trace:
  [bt] (0) /home/carles/.local/lib/python3.10/site-packages/xgboost/lib/libxgboost.so(+0x22dcbc) [0x7d1af182dcbc]
  [bt] (1) /home/carles/.local/lib/python3.10/site-packages/xgboost/lib/libxgboost.so(+0x52de3e) [0x7d1af1b2de3e]
  [bt] (2) /home/carles/.local/lib/python3.10/site-packages/xgboost/lib/libxgboost.so(+0x51b98d) [0x7d1af1b1b98d]
  [bt] (3) /home/carles/.local/lib/python3.10/site-packages/xgboost/lib/libxgboost.so(+0x51dbbc) [0x7d1af1b1dbbc]
  [bt] (4) /home/carles/.local/lib/python3.10/site-packages/xgboost/lib/libxgboost.so(+0x4cd1ca) [0x7d1af1acd1ca]
  [bt] (5) /home/carles/.local/lib/python3.10/site-packages/xgboost/lib/libxgboost.so(XGQuantileDMatrixCreateFromCallback+0x18c) [0x7d1af174556c]
  [bt] (6) /lib/x86_64-linux-gnu/libffi.so.8(+0x7e2e) [0x7d1b6aae9e2e]
  [bt] (7) 

## Testing

In [44]:
y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)

print(accuracy)

0.986
